# 3. Natural language processing for categorizing the extracted data

## Purpose, contents, & conclusions

**Purpose:** This notebook uses natural language processing to further clean the tabulated PDF data by grouping smells and flavors into broad categories. Each molecule is then assigned to one or more categories based on the reference books' descriptions.

**Contents:** The notebook contains:
* Basic natural language processing to clean, stem, and extract words potentially describing flavors or aromas (`flavor_neighbor_words` and `aroma_neighbor_words`, respectively) based on their proximity to indicator words like "scent" or "flavor".
* Broad categories and subcategories of flavors and smells are created, such as "floral" or "vegetal".
* Once stemmed, the potential flavor & aroma descriptors are assigned to or more categories / subcategories. For example, "rosy" is assigned to the category "floral" to create a translation dictionary or lexicon relevant to these reference books.
* Aroma and flavor categories are assigned for each molecule based on the parsed reference book descriptions. For example, the aroma of *Ε-Decalactone* (CAS number 5579-78-2) is assigned to the categories "starchy" and "dairy".

**Conclusions:** Key outputs are:
* `lexicon.csv`: A set of the 29 categories / subcategories used to describe both the flavor and the aroma of molecules in this data set.
* `stems-to-lexicon.csv`: A translation dictionary assigning stemmed descriptor words to one or more (sub) categories in the lexicon.
* The `categorized-descriptors` table is added to the `aromas-flavors.db` SQLite database. Each row is one molecule. The included data fields are a molecule's `name`, its unique `CAS_num` identifier, a list of categories describing its `aroma`, and a list of categories describing its `flavor`.
* The same data from the bullet point above is also exported to a tab-delimited CSV, `aromas-flavors-categorized.csv`.

## Explore a small subset of the data

The ultimate goal here is to extract words describing flavors and aromas from the full text of the description for each chemical. But how will that work? In this section, I will use a small subset of the data to explore some ideas. I'm limiting to only the first 100 entries because repeatedly iterating over the whole dataset while I am not sure what will work well would be slow and computationally expensive.

### Import a subset of the data

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
query = "SELECT name, CAS_num, full_description FROM fenaroli LIMIT 100"

In [3]:
with create_engine("sqlite:///aromas-flavors.db").connect() as db:
    molecules100 = pd.read_sql(query, db)

In [4]:
molecules100.head()

,name,CAS_num,full_description
0,Acacia Gum,9000-01-05,Botanical name: Acacia senegal (L ) Willd Bot...
1,Acetal,105-57-7,Synonyms: Acetaldehyde diethyl acetal; Acetald...
2,Acetaldehyde,75-07-0,Synonyms: Ethanal; Acetic aldehyde; Acetylalde...
3,"Acetaldehyde, Butyl Phenethyl Acetal",64577-91-9,"Synonyms: Benzene, (2-(1-butoxyethoxy)ethyl)-;..."
4,Acetaldehyde Di-Cis-3-Hexenyl Acetal,None,"Synonyms: 1,1-bis[3(Z)-Hexenyloxy]propane; (Z,..."


### Stemming vs. lemmatizing?

It will be important to pare down words to their roots so that, e.g. "citrusy" and "citrus" are both identified as the same sensation. Which is more appropriate here: stemming or lemmatizing?

Let's examine a few relevant pairs of sense words using a few different tools to compare the results.

In [5]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

In [6]:
porter = PorterStemmer()
snowball = SnowballStemmer("english")
lancaster = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

In [7]:
trial_senses = [("citrusy", "citrus"),
                ("tasting", "tastes"),
                ("herbal", "herbaceous"),
                ("cheese", "cheesy"),
                ("floral", "flower")]                

In [8]:
for pair in trial_senses:
    print("Input:\t\t", pair)
    print("Porter:\t\t", [porter.stem(word) for word in pair])
    print("Snowball:\t", [snowball.stem(word) for word in pair])
    print("Lancaster:\t", [lancaster.stem(word) for word in pair])
    print("Lemmas:\t\t", [lemmatizer.lemmatize(word) for word in pair])
    print("=====")

Input:		 ('citrusy', 'citrus')
Porter:		 ['citrusi', 'citru']
Snowball:	 ['citrusi', 'citrus']
Lancaster:	 ['citrusy', 'citr']
Lemmas:		 ['citrusy', 'citrus']
=====
Input:		 ('tasting', 'tastes')
Porter:		 ['tast', 'tast']
Snowball:	 ['tast', 'tast']
Lancaster:	 ['tast', 'tast']
Lemmas:		 ['tasting', 'taste']
=====
Input:		 ('herbal', 'herbaceous')
Porter:		 ['herbal', 'herbac']
Snowball:	 ['herbal', 'herbac']
Lancaster:	 ['herb', 'herbac']
Lemmas:		 ['herbal', 'herbaceous']
=====
Input:		 ('cheese', 'cheesy')
Porter:		 ['chees', 'cheesi']
Snowball:	 ['chees', 'cheesi']
Lancaster:	 ['chees', 'cheesy']
Lemmas:		 ['cheese', 'cheesy']
=====
Input:		 ('floral', 'flower')
Porter:		 ['floral', 'flower']
Snowball:	 ['floral', 'flower']
Lancaster:	 ['flor', 'flow']
Lemmas:		 ['floral', 'flower']
=====


All of these options look like they will take some finessing down the road -- ideally "citrusy" and "citrus" would be reduced to the same root for our purposes here. That said, the Snowball stemmer looks like the most reasonable approach.

### Clean the chemical descriptions

The descriptions contain a lot of information that is not useful in creating this database: a lot of numbers (we're only interested in descriptive words), a lot of punctuation, white spaces, etc. Clean up the descriptions by tokenizing the text, removing stopwords (like "at" or "and"), apply the Snowball stemmer, and remove extraneous characters like numbers or line breaks.

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [10]:
def desc_cleaner(row):
    """
    Called:
        By user.
    Purpose:
        Clean the a molecule's full description as read from a reference book.
        Tokenize the text, remove stopwords (like "at" or "and"), stem the words
        using a Snowball stemmer, and remove other non-descriptive information,
        such as numbers, symbols, linebreaks, and the word "like".
    Accepts:
        row: a row of a Pandas dataframe 
    Returns:
        a modified row of the dataframe
    """
    
    
    tokenized = word_tokenize(row["full_description"])
    nostops = [word.casefold() for word in tokenized if word not in set(stopwords.words("english"))]
    stems = [snowball.stem(word) for word in nostops]
    stems_str = " ".join(stems)
    no_likes = re.sub(r"like", " ", stems_str)
    alpha_chars_only = re.sub(r"[^a-zA-Z\s]", " ", no_likes)
    no_linebreaks = re.sub(r"\s+", " ", alpha_chars_only)
    row["clean_description"] = no_linebreaks
    return row

In [11]:
molecules100 = molecules100.apply(desc_cleaner, axis = "columns")

In [12]:
molecules100.head()

,name,CAS_num,full_description,clean_description
0,Acacia Gum,9000-01-05,Botanical name: Acacia senegal (L ) Willd Bot...,botan name acacia seneg l willd botan famili l...
1,Acetal,105-57-7,Synonyms: Acetaldehyde diethyl acetal; Acetald...,synonym acetaldehyd diethyl acet acetaldehyd e...
2,Acetaldehyde,75-07-0,Synonyms: Ethanal; Acetic aldehyde; Acetylalde...,synonym ethan acet aldehyd acetylaldehyd ethyl...
3,"Acetaldehyde, Butyl Phenethyl Acetal",64577-91-9,"Synonyms: Benzene, (2-(1-butoxyethoxy)ethyl)-;...",synonym benzen butoxyethoxi ethyl butoxy pheny...
4,Acetaldehyde Di-Cis-3-Hexenyl Acetal,None,"Synonyms: 1,1-bis[3(Z)-Hexenyloxy]propane; (Z,...",synonym bis z hexenyloxi propan z z ethylidene...


### Search for flavor and scent words

There is unfortunatley not a readily available list or dictionary of words used in the flavor & aroma industry. My research indicates that the descriptors (which the industry calls a *lexicon*) are not well standardized and vary between different sub-fields (such as wine vs. cheese vs. candy). Unfortunately, that means I will need to build my own lexicon here. It will not have the benefit of definition by professional flavorists or perfumists, but at least it will be somewhat general for the molecular data that I have available.

To build the lexicon, we'll want to identify descriptive words that appear near indicator words. For example, we want the phrase "with flavor notes of vanilla" to be captured as a vanilla-like flavor. To accomplish that, we'll define lists of `tags` that indicate a flavor or smell description.

In [13]:
flavor_tags = [snowball.stem(tag) for tag in ["flavor", "taste", "palate", "tongue", "mouth"]]
smell_tags = [snowball.stem(tag) for tag in ["smell", "scent", "aroma", "nose", "odor"]]

Define the `find_sensory` function to look for the tag words defined above, then grab the *neighboring* words within a set number of words (`window`) away from the tag word:

In [14]:
def find_sensory(row, tags: list, window: int, sense_colname: str):
    """
    Called:
        By user.
    Purpose:
        Find words that might describe a molecule's smell or flavor. To do so,
        iterate over the words in the clean_description column of a dataframe.
        When a word is found that is present in the tags list, append the
        *neighboring* +/- window words to sense_colname.

    Accepts:
        row: a row of a Pandas dataframe
        tags: a list of stemmed words that indicate a flavor or smell description
        window: an int, indicating how many words on either side of `tag` should
            be appended to the results that are found.
        sense_colname: a string naming a column of the dataframe where the
            potential flavor/smell words should be appended
    Returns:
        a modified row of the dataframe
    """
    
    desc = row["clean_description"].split(" ")
    senses = []
    for i, word in enumerate(desc):
        if word in tags:
            for neighbor_word in desc[i - window : i + window]:
                senses.append(neighbor_word)
    row[sense_colname] = " ".join(senses)
    return row

How far away from a `smell_tag` like "aroma" should we look for words that describe the flavor? That is, what value of `window` is most appropriate for use in the `find_sensory` function above? Let's try a few values and visually examine a few results.

In [15]:
for window_size in range(1, 10): 
    molecules100 = molecules100.apply(find_sensory, axis = "columns", 
                                      tags = smell_tags, 
                                      window = window_size, 
                                      sense_colname = ("smell-window-" + str(window_size)))

In [16]:
for row_num in range(0, 6):
    print("Name: ", molecules100.loc[row_num, "name"])
    for window_size in range(1, 10):
        print("--> window = ", window_size)
        print(molecules100.loc[row_num, ("smell-window-") + str(window_size)])
    print("\n===========\n")

Name:  Acacia Gum
--> window =  1
candi aroma
--> window =  2
hard candi aroma threshold
--> window =  3
pud hard candi aroma threshold valu
--> window =  4
gelatin pud hard candi aroma threshold valu n
--> window =  5
ice gelatin pud hard candi aroma threshold valu n a
--> window =  6
fruit ice gelatin pud hard candi aroma threshold valu n a usual
--> window =  7
juic fruit ice gelatin pud hard candi aroma threshold valu n a usual max
--> window =  8
fruit juic fruit ice gelatin pud hard candi aroma threshold valu n a usual max food
--> window =  9
dairi fruit juic fruit ice gelatin pud hard candi aroma threshold valu n a usual max food categori


Name:  Acetal
--> window =  1
green odor hcl aroma
--> window =  2
fruity green odor consumpt acid hcl aroma threshold
--> window =  3
pleasant fruity green odor consumpt annual miner acid hcl aroma threshold valu
--> window =  4
refresh pleasant fruity green odor consumpt annual lb amount miner acid hcl aroma threshold valu detect
--> windo

Comparing the output above with the original PDF documents, it looks like a `window` of 4 offers a good compromise for identifying aroma words: large enough to capture the relevant words, but not so large that additional convoluting words (such as those describing tastes instead of smells) are also captured.

Does the same apply for identifying flavor words? Repeat the procedure above, this time using the list of `flavor_tags` like "taste".

In [17]:
for window_size in range(1, 10): 
    molecules100 = molecules100.apply(find_sensory, axis = "columns", 
                                      tags = flavor_tags, 
                                      window = window_size, 
                                      sense_colname = ("flavor-window-" + str(window_size)))

In [18]:
for row_num in range(0, 6):
    print("Name: ", molecules100.loc[row_num, "name"])
    for window_size in range(1, 10):
        print("--> window = ", window_size)
        print(molecules100.loc[row_num, ("flavor-window-") + str(window_size)])
    print("\n===========\n")

Name:  Acacia Gum
--> window =  1
fixat flavor usual tast absolut tast
--> window =  2
food fixat flavor foam max usual tast threshold valu absolut tast characterist
--> window =  3
arab food fixat flavor foam stabil substitut max usual tast threshold valu threshold valu absolut tast characterist ppm
--> window =  4
gum arab food fixat flavor foam stabil beverag sugar substitut max usual tast threshold valu absolut tast threshold valu absolut tast characterist ppm spici
--> window =  5
use gum arab food fixat flavor foam stabil beverag adhes soup sugar substitut max usual tast threshold valu absolut tast usual tast threshold valu absolut tast characterist ppm spici sweet
--> window =  6
major use gum arab food fixat flavor foam stabil beverag adhes ice candi soup sugar substitut max usual tast threshold valu absolut tast characterist max usual tast threshold valu absolut tast characterist ppm spici sweet fruiti
--> window =  7
weight major use gum arab food fixat flavor foam stabil bev

Two things are clear from comparing the output above to the original PDF text:
1. In *Fenaroli's Handbook*, aroma descriptors are more common than flavor descriptors (at least in the first few entries).
1. When flavor descriptors are found, a `window` of 4 is again a reasonable compromise of finding relevant descriptors without searching too broadly. That said, some flavor descriptors were found quite far (`window = 9`, for Acadia Gum) from the tag word.

For building a lexicon, we'll use `window = 4` to pare down the number of possibilities found -- especially since there is going to be some manual processing involved at that step. When it comes time to actually assign molecules to flavor/aroma words from the lexicon, we'll use `window = 9` to ensure less is missed.

Make a set of all the words found in flavorlike_words:

In [19]:
combined_descriptions = ""
for desc in molecules100["flavor-window-4"].tolist():
    combined_descriptions += (desc + " ")

What does this set look like?

In [20]:
flavor_candidates = set(combined_descriptions.split(" "))
len(flavor_candidates)

279

In [21]:
flavor_candidates

{'',
 'a',
 'absolut',
 'acet',
 'acid',
 'acidul',
 'aconit',
 'acrid',
 'add',
 'addit',
 'adjuv',
 'agar',
 'agent',
 'alanin',
 'alaninemia',
 'allyl',
 'almond',
 'almost',
 'alo',
 'also',
 'ambrett',
 'amount',
 'and',
 'annual',
 'anthranil',
 'appar',
 'appl',
 'apricot',
 'aqueous',
 'arab',
 'aroma',
 'aromat',
 'as',
 'associ',
 'astring',
 'bacon',
 'balsam',
 'banana',
 'bay',
 'beanlik',
 'betel',
 'beverag',
 'bitter',
 'bland',
 'bouquet',
 'branch',
 'brandy',
 'brown',
 'burdock',
 'burnt',
 'butter',
 'butteri',
 'c',
 'cabbag',
 'candi',
 'caramel',
 'categori',
 'cereal',
 'charact',
 'characterist',
 'chees',
 'chemic',
 'cherri',
 'chip',
 'cider',
 'coconut',
 'color',
 'commerci',
 'compon',
 'concern',
 'confect',
 'consumpt',
 'cook',
 'corn',
 'cornmeal',
 'cream',
 'creami',
 'current',
 'delic',
 'deriv',
 'descript',
 'detect',
 'distinct',
 'dri',
 'earliest',
 'earthi',
 'enhanc',
 'equal',
 'especi',
 'essenti',
 'ethanol',
 'ether',
 'exampl',
 'exce

Great. This set looks reasonable; it contains many words that could describe a molecule's flavor, as well as many unrelated words.

## Build a lexicon

Having outlined and demonstrated a reasonable approach above, it's time to apply it to the entire data set to start building the lexicon.

### Import all the data & clean the descriptions

In [22]:
query = """SELECT name, CAS_num, full_description FROM fenaroli
           UNION
           SELECT name, CAS_num, full_description FROM 'common-materials'"""

In [23]:
with create_engine("sqlite:///aromas-flavors.db").connect() as db:
    molecules = pd.read_sql(query, db)

In [24]:
molecules = molecules.apply(desc_cleaner, axis = "columns")

In [25]:
molecules.head()

,name,CAS_num,full_description,clean_description
0,,"1-Methyl-2-(1,2,2-trimethyl[3.1.0]hex-3-ylmeth...","C15H26O, Mr 222.37, d20 4 0.947, is a colorles...",c h o mr d colorless viscous liquid power sand...
1,,1196-31-2,"*t = 20 (cid:1) C, **t = 4 (cid:1) C bp, ...",cid c cid c bp cid c
2,,12a],". In perfumery, acetates are the most importa...",in perfumeri acet import aliphat ester format...
3,,14073-97-3,,
4,(+)-Fenchone,7787-20-4,containing a small amount of the ( – ) isomer ...,contain small amount isom prepar dehydrogen fe...


It's already obvious that some errors are present. We already knew that some entries were missing a `name` or `CAS_num`. That doesn't matter at this stage. These errors will be corrected or removed later (after the lexicon is built).

### Find candidate aroma & flavor descriptors

In [26]:
molecules = molecules.apply(find_sensory, axis = "columns", 
                            tags = smell_tags, 
                            window = 4, 
                            sense_colname = "aroma_neighbor_words")

In [27]:
molecules = molecules.apply(find_sensory, axis = "columns", 
                            tags = flavor_tags, 
                            window = 4, 
                            sense_colname = "flavor_neighbor_words")

In [28]:
molecules.tail()

,name,CAS_num,full_description,clean_description,aroma_neighbor_words,flavor_neighbor_words
2995,Δ-Undecalactone,710-04-3,"Synonyms: 5-Hydroxy undecanoic acid, lactone; ...",synonym hydroxi undecano acid lacton n hexyl v...,undecalacton creami peach lik aroma consumpt a...,current level intak use flavor agent trade ass...
2996,Ε-Decalactone,5579-78-2,Synonyms: 6-Butylhexanolide; 7-Butyl-2-oxepano...,synonym butylhexanolid butyl oxepanon butyloxe...,sweet cream tonka bean odor consumpt annual lb...,current level intak use flavor agent trade ass...
2997,Ε-Dodecalactone,16429-21-3,Synonyms: epsilon-Dodecalactone; 7-Hexyl-2-oxe...,synonym epsilon dodecalacton hexyl oxepanon he...,fruiti peach apricot celeri odor consumpt annu...,current level intak use flavor agent trade ass...
2998,Ω-6-Hexadecenlactone,None,Synonyms: omega-6-Hexadecenlactone; Oxacyclohe...,synonym omega hexadecenlacton oxacycloheptadec...,hexadecenlacton strong musk lik odor consumpt ...,current level intak use flavor agent trade ass...
2999,Ω-Pentadecalactone,106-02-5,Synonyms: Angelica lactone; Exaltolide; ω-Lact...,synonym angelica lacton exaltolid lacton penta...,extraordinarili persist musk lik odor nas no c...,current level intak use flavor agent trade ass...


Create sets of words that potentially describe taste (`flavor_candidates`) or smell (`aroma_candidates`)

In [29]:
flavor_descriptions = ""
for desc in molecules["flavor_neighbor_words"].tolist():
    flavor_descriptions += (desc + " ")
flavor_candidates = set(flavor_descriptions.split(" "))
print("Count of words potentially describing flavor: ", len(flavor_candidates))

Count of words potentially describing flavor:  1619


In [30]:
aroma_descriptions = ""
for desc in molecules["aroma_neighbor_words"].tolist():
    aroma_descriptions += (desc + " ")
aroma_candidates = set(aroma_descriptions.split(" "))
print("Count of words potentially describing flavor: ", len(aroma_candidates))

Count of words potentially describing flavor:  2853


Great. A soup of ~4400 words potentially describing flavor or aroma were found.

### Sort the candidate descriptors

This is going to be ugly, but again, I can't find a pre-built list or dictionary of standardized words describing flavors or aromas. So, we'll have to sort the words that from `flavor_candidates` and `aroma_candidates` manually.

I'm going to sort these sets manually in a spreadsheet. So, export the two sets to a CSV:

In [31]:
import csv

In [32]:
with open("work-in-progress/sense_word_candidates.csv", "w") as file:
    filewriter = csv.writer(file, delimiter = ",")
    for candidate in flavor_candidates.union(aroma_candidates):
        filewriter.writerow([candidate])

**>> We'll take a brief intermission here while I go sort the spreadsheet. Back in a flash.**

In [33]:
from IPython.display import Image

In [34]:
Image(url="https://freesvg.org/storage/img/thumb/1540863971.png")

**...**

**>> Okay, great. All done.**

### Review the lexicon

Now that the `flavor_candidates` and `aroma_candidates` have been manually sorted and assigned to broad categories (and subcategories!), we have a lexicon of basic words describing flavors and smells. A proper flavorist or perfumist would doubtless have a more appropriate set of categories, but as mentioned above, I could not find a universal lexicon. This will have to do.

In [35]:
lexicon = pd.read_csv("lexicon.csv")
lexicon

,category,subcategory
0,brown,NaN
1,brown,chocolate
2,brown,vanilla
3,earthy,NaN
4,earthy,dry
5,earthy,fungal
6,earthy,nutty
7,earthy,woody
8,fruity,NaN
9,fruity,berry


I also created a dictionary that assigns each stemmed descriptor to one or more categories / subcategories:

In [36]:
stem_translator = {}
with open("stems-to-lexicon.csv") as file:
    reader = csv.DictReader(file)
    for row in reader:
        stem = row["word_stem"]
        desc = ""
        for i in [1, 2, 3]:
            if row["Category_" + str(i)]:
                desc +=row ["Category_" + str(i)]
                if row["Subcategory_" + str(i)]:
                    desc += ("|" + row["Subcategory_" + str(i)])
                desc += " "
        stem_translator[stem] = desc.rstrip()

In [37]:
stem_translator["apple"]

'fruity'

In [38]:
stem_translator["toast"]

'starchy|bready savory|cooked'

## Assign molecules to categories of flavors & smells

Now it's time to put everything together! Using the `find_sensory` function defined above, I'll search through the `aroma_neighbor_words` and `flavor_neighbor_words` columns of the dataframe to find `descriptors` that have a categorical definition in the new lexicon. 

In [39]:
def assign_sense_categories(row, search_column: str, result_column: str, sensory_dict: dict):
    """
    Called:
        By user.
    Purpose:
        Iterate over the words in search_column. When one is found that is
        defined in sensory_dict, append that definition to result_column.
    Accepts:
        row: a row of a Pandas dataframe
        search_column: a string that is the name of a column in the dataframe
        result_column: a string that is the name of a column in the dataframe
        sensory_dict: a dict translating stemmed words into sensory categories
    Returns:
        a modified row of the dataframe
    """
    
    translation = set()
    
    # Search the dict for translations of the stemmed word
    for word in row[search_column].split(" "):
        if word in sensory_dict.keys():
            translation.update(sensory_dict[word].split(" "))
    
    # If no translation is found, return None instead.
    if len(translation) > 0:
        row[result_column] = " ".join(translation)
    else:
        row[result_column] = None
       
    return row

Apply the function for categorizing aroma words:

In [40]:
molecules = molecules.apply(assign_sense_categories,
                            search_column = "aroma_neighbor_words",
                            result_column = "aroma",
                            sensory_dict = stem_translator,
                            axis = "columns")

And also for flavor words:

In [41]:
molecules = molecules.apply(assign_sense_categories,
                            search_column = "flavor_neighbor_words",
                            result_column = "flavor",
                            sensory_dict = stem_translator,
                            axis = "columns")

In [42]:
molecules

,name,CAS_num,full_description,clean_description,aroma_neighbor_words,flavor_neighbor_words,aroma,flavor
0,,"1-Methyl-2-(1,2,2-trimethyl[3.1.0]hex-3-ylmeth...","C15H26O, Mr 222.37, d20 4 0.947, is a colorles...",c h o mr d colorless viscous liquid power sand...,liquid power sandel wood odor creami rosi unde...,,floral earthy|woody,None
1,,1196-31-2,"*t = 20 (cid:1) C, **t = 4 (cid:1) C bp, ...",cid c cid c bp cid c,,,None,None
2,,12a],". In perfumery, acetates are the most importa...",in perfumeri acet import aliphat ester format...,ester prefer artifici fruit aroma natur acet e...,use conif note in flavor composit aliphat ester,fruity,earthy|woody
3,,14073-97-3,,,,,None,None
4,(+)-Fenchone,7787-20-4,containing a small amount of the ( – ) isomer ...,contain small amount isom prepar dehydrogen fe...,,obtain cid individu fragranc flavor materi alo...,None,None
...,...,...,...,...,...,...,...,...
2995,Δ-Undecalactone,710-04-3,"Synonyms: 5-Hydroxy undecanoic acid, lactone; ...",synonym hydroxi undecano acid lacton n hexyl v...,undecalacton creami peach lik aroma consumpt a...,current level intak use flavor agent trade ass...,fruity,fruity brown fruity|berry
2996,Ε-Decalactone,5579-78-2,Synonyms: 6-Butylhexanolide; 7-Butyl-2-oxepano...,synonym butylhexanolid butyl oxepanon butyloxe...,sweet cream tonka bean odor consumpt annual lb...,current level intak use flavor agent trade ass...,starchy dairy,None
2997,Ε-Dodecalactone,16429-21-3,Synonyms: epsilon-Dodecalactone; 7-Hexyl-2-oxe...,synonym epsilon dodecalacton hexyl oxepanon he...,fruiti peach apricot celeri odor consumpt annu...,current level intak use flavor agent trade ass...,fruity vegetal|vegetable,None
2998,Ω-6-Hexadecenlactone,None,Synonyms: omega-6-Hexadecenlactone; Oxacyclohe...,synonym omega hexadecenlacton oxacycloheptadec...,hexadecenlacton strong musk lik odor consumpt ...,current level intak use flavor agent trade ass...,floral earthy|woody,vegetal fruity|berry


*Fantastic.* It looks like the flavor & smell descriptors have been appropriately parsed & categorized into the `aroma` and `flavor` columns.

## Clean up the resulting dataframe

The `molecules` dataframe contains all the useful information we need, but also a lot of unusable fluff. Some rows are missing a `CAS_num`, and others have neither an `aroma` or `flavor` that are characterized. Create a copy of the `molecules` dataframe to clean up:

In [43]:
keep_cols = ["name", "CAS_num", "aroma", "flavor"]
aromas_flavors = molecules[keep_cols]

### Remove rows missing critical values

Remove the rows that have neither an `aroma` nor a `flavor`:

In [44]:
aromas_flavors = aromas_flavors[pd.notna(aromas_flavors["flavor"]) | pd.notna(aromas_flavors["aroma"])]
aromas_flavors

,name,CAS_num,aroma,flavor
0,,"1-Methyl-2-(1,2,2-trimethyl[3.1.0]hex-3-ylmeth...",floral earthy|woody,None
2,,12a],fruity,earthy|woody
5,(+)-Menthofuran,17957-94-7,fruity floral fatty fruity|citrusy,None
7,(+/-)-1-Hepten-3-Ol,4938-52-7,fatty dairy,vegetal fruity sharp|chemical liquorlike
8,"(+/-)-2,4,8-Trimethyl-7-Nonen-2-Ol",437770-28-0,fruity fruity|citrusy,fruity|berry fruity|citrusy
...,...,...,...,...
2995,Δ-Undecalactone,710-04-3,fruity,fruity brown fruity|berry
2996,Ε-Decalactone,5579-78-2,starchy dairy,None
2997,Ε-Dodecalactone,16429-21-3,fruity vegetal|vegetable,None
2998,Ω-6-Hexadecenlactone,None,floral earthy|woody,vegetal fruity|berry


Also remove the rows that have neither a `name` or a `CAS_num`.

In [45]:
def invalid_to_na(row, colname: str, min_len: int, max_len: int):
    """
    Called:
        By user.
    Purpose:
        Check the validity of the value in the `colname` column of a dataframe
        by ensuring its length falls between minimum and maximum limits.
    Accepts:
        row: a row of a Pandas dataframe
        colname: a string that is the name of a column in the dataframe
        min_len: an int defining the minimum acceptable length of the string
            data in colname
        max_len: an int defining the maximum acceptable length of the string
            data in colname
    Returns:
        a modified row of the dataframe
    """
    
    if row[colname] and len(row[colname]) > min_len and len(row[colname]) < max_len:
        pass
    else:
        row[colname] = None
    
    return row

In [46]:
aromas_flavors = aromas_flavors.apply(invalid_to_na, colname = "name", min_len = 5, max_len = 256, axis = "columns")
aromas_flavors = aromas_flavors.apply(invalid_to_na, colname = "CAS_num", min_len = 7, max_len = 15, axis = "columns")
aromas_flavors = aromas_flavors[pd.notna(aromas_flavors["name"]) | pd.notna(aromas_flavors["CAS_num"])]
aromas_flavors

,name,CAS_num,aroma,flavor
5,(+)-Menthofuran,17957-94-7,fruity floral fatty fruity|citrusy,None
7,(+/-)-1-Hepten-3-Ol,4938-52-7,fatty dairy,vegetal fruity sharp|chemical liquorlike
8,"(+/-)-2,4,8-Trimethyl-7-Nonen-2-Ol",437770-28-0,fruity fruity|citrusy,fruity|berry fruity|citrusy
9,"(+/-)-2,8-Epithio-Cis-P-Menthane",68398-18-5,earthy fruity|citrusy,vegetal fruity sharp|sulfurous fruity|citrusy
10,(+/-)-2-Hydroxypiperitone,490-03-9,vegetal fruity|berry vegetal|herbal,fruity
...,...,...,...,...
2995,Δ-Undecalactone,710-04-3,fruity,fruity brown fruity|berry
2996,Ε-Decalactone,5579-78-2,starchy dairy,None
2997,Ε-Dodecalactone,16429-21-3,fruity vegetal|vegetable,None
2998,Ω-6-Hexadecenlactone,None,floral earthy|woody,vegetal fruity|berry


### Find missing `name` from `CAS_num`, or vice versa

Are there many entries missing either piece of data?

In [47]:
print("Entries missing a name: {}".format(aromas_flavors["name"].isna().sum()))

Entries missing a name: 16


In [48]:
print("Entries missing a CAS number: {}".format(aromas_flavors["CAS_num"].isna().sum()))

Entries missing a CAS number: 756


In [49]:
import requests
from time import sleep

In [50]:
# In this generic query, the first blank is the structural identifier (CAS or name), and the second blank is the desired output ("smiles" or "inchi")
query_url = "https://cactus.nci.nih.gov/chemical/structure/{}/{}"

def name_cas_lookup(row):
    """
    Called:
        By user.
    Purpose:
        Check a dataframe row for missing CAS number (`CAS_num` column) or a
        missing name (`name` column). When one is missing, use the other
        to retrieve it from the NIH CACTUS API.
    Accepts:
        row: a row of a Pandas dataframe
    Returns:
        a modified row of the dataframe
    """
    
    
    # If no CAS number is found, look it up by name
    if pd.isna(row["CAS_num"]):
        try:
            reply = requests.get(query_url.format(row["name"], "cas"), timeout = 5)
            if reply.status_code == 200:
                row["CAS_num"] = reply.text
                sleep(1)
        except:
            pass
    
    # If no name is found, look it up by CAS number
    if pd.isna(row["name"]):
        try:
            reply = requests.get(query_url.format(row["name"], "iupac_name"), timeout = 4)
            if reply.status_code == 200:
                row["name"] = reply.text
                sleep(1)
        except:
            pass

    return row

In [51]:
aromas_flavors = aromas_flavors.apply(name_cas_lookup, axis = "columns")

Are there any remaining entries missing a CAS number or name?

In [52]:
print("Entries still missing a name: {}".format(aromas_flavors["name"].isna().sum()))
print("Entries still missing a CAS number: {}".format(aromas_flavors["CAS_num"].isna().sum()))

Entries still missing a name: 16
Entries still missing a CAS number: 451


While it is unfortunate that CAS numbers were not found for more of these entries, let's move ahead by keeping them. It may yet be possible to salvage the data for those entries through other means.

## Write the data to the database

We'll write the new categorical data to the `categorized-descriptors` table of the `aromas-flavors.db` SQLite database.

In [56]:
with create_engine("sqlite:///aromas-flavors.db").connect() as db:
    aromas_flavors.to_sql("categorized-descriptors", db, index = False, if_exists = "replace")

## Export data as a CSV

For sharing purposes, it would also be nice to export the data as a CSV file. Because some of the chemical names include commas, I'll use a tab instead of a comma as the delimiter.

In [58]:
aromas_flavors.to_csv("aromas-flavors-categorized.csv", sep = "\t", index = False)